# Real Time Video Demo

## Importing Required Libraries

In [1]:
import cv2
import numpy as np
import tensorflow as tf
import time
import winsound  # For Windows beep alert

## Load the trained model And Defining Class Labels

In [2]:
new_model = tf.keras.models.load_model('models/MyModel1.h5')
class_names = ['Close-Eyes', 'Open-Eyes']

## Set sound frequency and duration

In [10]:
frequency = 2500  # Hz
duration = 2000   # 2 seconds

## Load Haarcascades for face and eyes (including eyeglasses)

In [8]:
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier("haarcascade_eye.xml")  # Normal eye cascade
eyeglasses_cascade = cv2.CascadeClassifier("haarcascade_eye_tree_eyeglasses.xml")  # Eyeglasses-friendly cascade

## Open webcam For A Real-Time Video Demo
- The code captures video from the webcam.
- Detects faces and eyes using OpenCV.
- Uses a deep learning model to classify eye state.
- Alerts the user if no face is detected for 3 seconds.
- Triggers a sleep alert if eyes are closed for too long.

In [ ]:
# Open webcam
cap = cv2.VideoCapture(0)
cap.set(3, 400)  # Set width
cap.set(4, 500)  # Set height

if not cap.isOpened():
    raise IOError("Cannot Open Webcam")

counter = 3  # Counter for closed eyes
status = "Open Eyes"
face_detected_time = time.time()  # Track time since last face detection

while True:
    ret, frame = cap.read()
    if not ret:
        break  # Exit if frame not captured

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    if len(faces) == 0:
        # If no face detected for more than 3 seconds, alert
        if time.time() - face_detected_time > 3:
            cv2.putText(frame, 'Face Missing Alert!', (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            winsound.Beep(frequency, duration)  # Play alert sound
    else:
        face_detected_time = time.time()  # Reset face detected timer

    eyes_detected = False  # Flag to check if eyes are detected

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        # Draw face rectangle
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 0), 2)

        # Resize face for model prediction
        face_resized = cv2.resize(roi_color, (224, 224))  # Model expects 224x224
        face_resized = np.expand_dims(face_resized, axis=0)  # Add batch dimension

        # Predict eye state
        prediction = new_model.predict(face_resized)
        predicted_class = class_names[np.argmax(prediction[0])]
        status = predicted_class

        # First, try detecting eyes with the normal cascade
        eyes = eye_cascade.detectMultiScale(roi_gray, 1.1, 5)

        # If no eyes detected, try detecting with eyeglasses cascade
        if len(eyes) == 0:
            eyes = eyeglasses_cascade.detectMultiScale(roi_gray, 1.1, 5)

        if len(eyes) > 0:
            eyes_detected = True
            for (ex, ey, ew, eh) in eyes:
                cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)

    # If no eyes detected, assume eyes are closed
    if not eyes_detected:
        counter += 1
        status = "Closed Eyes"
        color = (0, 0, 255)  # Red frame
    else:
        counter = 0  # Reset counter when eyes are open
        status = "Open Eyes"
        color = (0, 255, 0)  # Green frame

    # Draw status text
    cv2.putText(frame, status, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    # Alert if eyes are closed for too long
    if counter > 5:
        cv2.putText(frame, 'Sleep Alert !!', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        winsound.Beep(frequency, duration)  # Play beep sound on Windows
        counter = 0  # Reset counter after alert

    # Show the frame
    cv2.imshow("Drowsiness Detection", frame)

    # Break the loop when 'Esc' key is pressed
    if cv2.waitKey(1) & 0xFF == 27:  # 27 is the ASCII code for 'Esc'
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
